# 실습 세션 1: LLM 및 프롬프트 엔지니어링 활용

본 세션에서는 Azure AI Foundry를 사용하여 LLM을 배포하고, Chat Playground와 Jupyter Notebook에서 다양한 프롬프트 엔지니어링 기법을 실습합니다.

## 목표
- Azure AI Foundry에서 LLM 모델 배포하기
- 프롬프트 엔지니어링 기본 기법 학습하기
- 다양한 프롬프트 패턴 실습하기

## 1. Azure OpenAI 연결 설정

먼저 필요한 패키지를 설치하고 Azure OpenAI 서비스에 연결하기 위한 설정을 진행합니다.

In [3]:
# 필요한 라이브러리 설치
!pip install -r requirements.txt

  Using cached autogen_agentchat-0.5.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached openai-1.76.0-py3-none-any.whl.metadata (25 kB)
  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tiktoken-0.9.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached autogen_ext-0.5.5-py3-none-any.whl.metadata (7.0 kB)
  Using cached autogen_core-0.5.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached jsonref-1.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached opentelemetry_api-1.32.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached pillow-11.2.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (8.9 kB)
  Using cached protobuf-5.29.4-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metada

In [4]:
import os
import json
from dotenv import load_dotenv
from openai import AzureOpenAI

# .env 파일에서 환경 변수 로드 (없는 경우 직접 입력)
load_dotenv()

# Azure OpenAI 설정
api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")

# AzureOpenAI 클라이언트 생성
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version=api_version
)

# 모델 배포 이름
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-35-turbo") # 배포된 모델 이름

## 2. Azure AI Foundry에서 LLM 모델 배포 확인

Azure AI Foundry(이전의 Azure OpenAI Service)에서 배포한 LLM 모델 정보를 확인해봅시다.

In [5]:
# 배포된 모델 목록 확인
try:
    # 참고: 현재 버전의 AzureOpenAI 클라이언트에서는 
    # 모든 배포 목록을 직접 확인하는 기능이 제한적일 수 있습니다.
    # 아래 코드는 예시일 뿐이며, 실제 환경에 따라 다를 수 있습니다.
    print("배포된 모델 정보:")
    print(f"- 현재 사용 중인 모델: {deployment_name}")
    print("Azure Portal에서 배포된 모든 모델을 확인할 수 있습니다.")

    # 배포된 모델 테스트 (간단한 테스트)
    test_response = client.chat.completions.create(
        model=deployment_name,
        messages=[{"role": "user", "content": "안녕하세요"}],
        max_tokens=10
    )
    print(f"\n모델 응답 테스트 성공: {test_response.choices[0].message.content}")
except Exception as e:
    print(f"모델 확인 중 오류 발생: {e}")
    print("Azure Portal에서 직접 배포 상태를 확인해주세요.")

배포된 모델 정보:
- 현재 사용 중인 모델: gpt-4o-mini
Azure Portal에서 배포된 모든 모델을 확인할 수 있습니다.

모델 응답 테스트 성공: 안녕하세요! 어떻게 도와드릴까요?


## 3. 기본 프롬프트 엔지니어링

프롬프트 엔지니어링의 기본 기법들을 실습해봅시다.

In [6]:

def get_completion(prompt, model=deployment_name):
    """지정된 프롬프트와 모델을 사용하여 Azure OpenAI API에 요청을 보내고 응답을 반환합니다."""
    # 메시지 구성
    messages = [
        {"role": "user", "content": prompt}
    ]
    
    # 채팅 완성 요청
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=4096
    )
    
    # 응답에서 메시지 내용 추출
    return response.choices[0].message.content

# 간단한 프롬프트 테스트
simple_prompt = "Azure란 무엇인가요?"
print(get_completion(simple_prompt))

Azure는 Microsoft에서 제공하는 클라우드 컴퓨팅 플랫폼입니다. Azure는 다양한 클라우드 서비스와 솔루션을 제공하여 기업과 개발자들이 애플리케이션을 구축, 배포 및 관리할 수 있도록 지원합니다. Azure의 주요 기능 및 구성 요소는 다음과 같습니다:

1. **컴퓨팅 서비스**: 가상 머신, 컨테이너, 서버리스 컴퓨팅 등 다양한 컴퓨팅 리소스를 제공하여 사용자가 필요에 맞게 애플리케이션을 실행할 수 있습니다.

2. **스토리지 서비스**: Blob 스토리지, 파일 스토리지, 데이터베이스 서비스 등을 통해 데이터를 안전하게 저장하고 관리할 수 있습니다.

3. **데이터베이스**: Azure SQL Database, Cosmos DB, Azure Database for PostgreSQL 등 다양한 데이터베이스 솔루션을 제공합니다.

4. **네트워킹**: 가상 네트워크, 로드 밸런서, VPN Gateway 등 네트워크 관련 서비스로 안전하고 효율적인 연결을 지원합니다.

5. **AI 및 머신러닝**: Azure는 AI 및 머신러닝 서비스도 제공하여, 개발자가 인공지능 기반의 애플리케이션을 쉽게 개발할 수 있도록 도와줍니다.

6. **IoT**: IoT Hub와 같은 서비스를 통해 사물인터넷(IoT) 솔루션을 구축하고 관리할 수 있습니다.

7. **보안 및 관리**: Azure는 데이터 보호 및 보안을 위한 다양한 도구와 서비스를 제공하여 사용자가 클라우드 환경에서 안전하게 작업할 수 있도록 지원합니다.

Azure는 유연한 요금제와 다양한 서비스 모델을 제공하여 기업이 필요에 따라 리소스를 쉽게 조정할 수 있도록 해줍니다. 이로 인해 많은 기업들이 Azure를 클라우드 솔루션으로 선택하고 있습니다.


### 3.1 명확한 지시사항 제공하기

LLM에게 명확한 지시사항을 제공하면 더 좋은 결과를 얻을 수 있습니다.

In [7]:
# 명확하지 않은 지시사항
unclear_prompt = "클라우드 컴퓨팅에 대해 알려줘."
print("\n=== 명확하지 않은 지시사항 ===\n")
print(get_completion(unclear_prompt))

# 명확한 지시사항
clear_prompt = """클라우드 컴퓨팅에 대해 다음 형식으로 설명해줘:
1. 정의 (50단어 이내)
2. 주요 이점 (5가지 나열)
3. 주요 서비스 제공업체 (3개)
4. Azure의 주요 서비스 (5가지)
"""
print("\n=== 명확한 지시사항 ===\n")
print(get_completion(clear_prompt))


=== 명확하지 않은 지시사항 ===

클라우드 컴퓨팅(Cloud Computing)은 인터넷(클라우드)을 통해 서버, 스토리지, 데이터베이스, 네트워킹, 소프트웨어, 분석 등 컴퓨팅 리소스를 제공하는 방식입니다. 이를 통해 사용자는 직접 하드웨어를 구매하거나 관리할 필요 없이, 필요할 때 필요한 만큼의 리소스를 사용할 수 있습니다. 클라우드 컴퓨팅의 주요 특징과 장점은 다음과 같습니다.

### 주요 특징

1. **온디맨드 셀프 서비스**: 사용자는 필요할 때 언제든지 리소스를 요청하고 사용할 수 있습니다.
   
2. **광범위한 네트워크 액세스**: 인터넷이 가능한 모든 장치(노트북, 스마트폰 등)에서 접근할 수 있습니다.

3. **자원 풀링**: 여러 사용자들이 공유하는 리소스를 통해 효율적인 자원 관리가 가능합니다.

4. **빠른 탄력성**: 필요에 따라 리소스를 신속하게 확장하거나 축소할 수 있습니다.

5. **측정된 서비스**: 사용량에 따라 비용이 청구되며, 사용자는 실제 사용한 만큼만 비용을 지불합니다.

### 클라우드 서비스 모델

클라우드 컴퓨팅은 일반적으로 세 가지 주요 서비스 모델로 나뉩니다.

1. **IaaS (Infrastructure as a Service)**: 서버, 스토리지, 네트워크 등 기본 인프라를 제공하는 서비스입니다. 사용자는 운영 체제와 애플리케이션을 설치하고 관리할 수 있습니다. 예: Amazon EC2, Google Compute Engine.

2. **PaaS (Platform as a Service)**: 애플리케이션을 개발, 실행, 관리할 수 있는 플랫폼을 제공합니다. 개발자는 인프라를 관리할 필요 없이 애플리케이션 개발에 집중할 수 있습니다. 예: Google App Engine, Microsoft Azure.

3. **SaaS (Software as a Service)**: 소프트웨어 애플리케이션을 인터넷을 통해 제공하는 서비스입니다. 사용자는 소프트웨어를 설치할 필요 없이 웹 브라우저를 

### 3.2. 역할 부여하기 (Role Prompting)

LLM에게 특정 역할을 부여하면 해당 관점에서 응답을 생성할 수 있습니다.

In [8]:
# 역할 없는 프롬프트
no_role_prompt = "Azure Kubernetes Service(AKS)의 장점을 설명해줘."
print("\n=== 역할 없는 프롬프트 ===\n")
print(get_completion(no_role_prompt))

# 역할 부여 프롬프트
role_prompt = """당신은 5년 경력의 클라우드 아키텍트입니다. 
기업의 IT 관리자에게 Azure Kubernetes Service(AKS)를 도입해야 하는 이유와 장점을 
기술적 측면과 비즈니스 측면에서 설득력 있게 설명해주세요."""
print("\n=== 역할 부여 프롬프트 ===\n")
print(get_completion(role_prompt))


=== 역할 없는 프롬프트 ===

Azure Kubernetes Service(AKS)는 Microsoft Azure에서 제공하는 관리형 Kubernetes 서비스로, 여러 가지 장점을 가지고 있습니다. 다음은 AKS의 주요 장점입니다:

1. **관리형 서비스**: AKS는 Kubernetes 클러스터의 배포, 관리 및 운영을 간소화합니다. 사용자는 인프라 관리에 대한 부담을 덜고 애플리케이션 개발에 집중할 수 있습니다.

2. **자동 스케일링**: AKS는 수요에 따라 자동으로 노드를 추가하거나 제거할 수 있는 기능을 제공합니다. 이를 통해 리소스를 효율적으로 사용할 수 있고, 비용을 절감할 수 있습니다.

3. **통합된 Azure 서비스**: AKS는 Azure의 다른 서비스와 원활하게 통합됩니다. Azure Monitor, Azure Active Directory, Azure DevOps 등과 통합하여 모니터링, 인증 및 CI/CD 파이프라인을 쉽게 구성할 수 있습니다.

4. **고가용성**: AKS는 여러 지역에 걸쳐 클러스터를 배포할 수 있어, 장애 발생 시에도 높은 가용성을 유지할 수 있습니다.

5. **보안**: Azure의 보안 기능을 활용하여 클러스터를 보호할 수 있습니다. Azure Active Directory 통합, 네트워크 보안 그룹, Azure Policy 등을 통해 보안을 강화할 수 있습니다.

6. **신속한 배포**: AKS는 Kubernetes 애플리케이션을 신속하게 배포하고 관리할 수 있는 도구들을 제공합니다. Helm 차트와 같은 패키지 관리 도구를 사용하여 애플리케이션을 쉽게 배포할 수 있습니다.

7. **커뮤니티 지원**: Kubernetes는 오픈 소스 프로젝트로서, 방대한 커뮤니티와 생태계가 있습니다. AKS를 사용하면 이러한 커뮤니티의 다양한 도구와 리소스를 활용할 수 있습니다.

8. **비용 효율성**: AKS는 사용한 만큼만 비용을 지불하는 모델을 제공하여, 초기 투자 비용을 줄이고 필요한 만큼만

### 3.3. 단계별 사고 유도 (Chain-of-Thought)

복잡한 문제를 해결할 때는 LLM에게 단계별로 생각하도록 유도하는 것이 효과적입니다.

In [9]:
# 직접적인 질문
direct_prompt = "Azure에서 3-tier 웹 애플리케이션을 구축하는 방법을 알려줘."
print("\n=== 직접적인 질문 ===\n")
print(get_completion(direct_prompt))

# 단계별 사고 유도 프롬프트
cot_prompt = """Azure에서 3-tier 웹 애플리케이션을 구축하려고 합니다. 
다음 단계에 따라 설계 방법을 설명해주세요:

1. 먼저, 각 계층(프레젠테이션, 비즈니스 로직, 데이터)에 적합한 Azure 서비스를 선택하세요.
2. 다음으로, 이러한 서비스들이 어떻게 연결되는지 설명하세요.
3. 그 다음, 보안 측면에서 고려해야 할 사항을 설명하세요.
4. 마지막으로, 확장성을 위한 설계 방법을 제안하세요.

각 단계별로 구분하여 자세히 설명해주세요."""
print("\n=== 단계별 사고 유도 프롬프트 ===\n")
print(get_completion(cot_prompt))


=== 직접적인 질문 ===

Azure에서 3-tier 웹 애플리케이션을 구축하는 방법에 대해 설명드리겠습니다. 3-tier 아키텍처는 일반적으로 프레젠테이션 계층, 비즈니스 로직 계층, 데이터 계층으로 구성됩니다. 다음은 Azure에서 이 아키텍처를 구현하기 위한 단계입니다.

### 1. Azure 계정 생성
Azure에서 리소스를 사용하려면 Azure 계정이 필요합니다. [Azure 웹사이트](https://azure.microsoft.com/)에서 계정을 생성하세요.

### 2. 리소스 그룹 생성
모든 리소스를 관리하기 위해 리소스 그룹을 생성합니다.

- Azure Portal에 로그인합니다.
- "리소스 그룹"을 선택하고 "추가"를 클릭합니다.
- 리소스 그룹의 이름과 지역을 설정한 후 생성합니다.

### 3. 프레젠테이션 계층 (Web Tier)
프레젠테이션 계층은 사용자 인터페이스를 제공합니다. Azure App Service를 사용해 웹 애플리케이션을 호스팅할 수 있습니다.

- Azure Portal에서 "App Services"를 선택하고 "추가"를 클릭합니다.
- 애플리케이션의 이름, 구독, 리소스 그룹, 운영 체제 및 가격 책정을 선택하여 애플리케이션을 만듭니다.

### 4. 비즈니스 로직 계층 (Middle Tier)
비즈니스 로직을 처리하는 API를 구현합니다. Azure Functions 또는 Azure App Service Web API를 사용할 수 있습니다.

- Azure Portal에서 "Function App" 또는 "App Service"를 선택하여 새로운 API를 생성합니다.
- 필요한 언어와 프레임워크를 선택하여 비즈니스 로직을 구현합니다.

### 5. 데이터 계층 (Data Tier)
데이터를 저장할 데이터베이스를 설정합니다. Azure SQL Database 또는 Cosmos DB를 사용할 수 있습니다.

- Azure Portal에서 "SQL Database" 또는 "Cosmos DB"를 선택하여 데이

### 3.4. Few-Shot 학습

몇 가지 예시를 제공하여 LLM이 패턴을 학습하도록 유도할 수 있습니다.

In [10]:
# Few-shot 프롬프트
few_shot_prompt = """다음은 클라우드 서비스에 대한 간결한 설명입니다:

서비스: Azure Virtual Machine
설명: 다양한 크기와 운영체제를 지원하는 확장 가능한 컴퓨팅 리소스로, IaaS 솔루션을 제공합니다.

서비스: AWS Lambda
설명: 서버리스 컴퓨팅 서비스로, 코드 실행만 관리하며 인프라 관리가 필요 없는 이벤트 기반 애플리케이션을 구축할 수 있습니다.

서비스: Google Cloud Storage
설명: 확장 가능하고 내구성 있는 객체 스토리지 솔루션으로, 정적 콘텐츠 호스팅부터 빅데이터 분석까지 다양한 용도로 활용됩니다.

위의 형식을 따라 아래 서비스에 대한 설명을 작성해주세요:

서비스: Azure Kubernetes Service (AKS)
설명:

서비스: Azure Functions
설명:

서비스: Azure Cosmos DB
설명:
"""
print(get_completion(few_shot_prompt))

서비스: Azure Kubernetes Service (AKS)  
설명: Kubernetes 클러스터를 쉽게 관리할 수 있는 서비스로, 컨테이너화된 애플리케이션의 배포, 관리 및 확장을 지원합니다.

서비스: Azure Functions  
설명: 이벤트 기반 서버리스 컴퓨팅 플랫폼으로, 코드 실행을 트리거하는 이벤트에 따라 자동으로 확장되며 인프라 관리 없이 비즈니스 로직을 구현할 수 있습니다.

서비스: Azure Cosmos DB  
설명: 글로벌 분산형 다중 모델 데이터베이스 서비스로, 높은 가용성과 빠른 응답 속도를 제공하며, 다양한 데이터 모델을 지원하여 애플리케이션의 요구에 맞게 사용할 수 있습니다.


## 4. 고급 프롬프트 엔지니어링 기법

보다 복잡한 작업을 수행하기 위한 고급 프롬프트 엔지니어링 기법을 살펴봅시다.

### 4.1. 시스템 메시지 활용하기

시스템 메시지를 사용하여 LLM의 전반적인 행동을 정의할 수 있습니다.

In [11]:
def get_completion_with_system(system_message, user_message, model=deployment_name):
    """시스템 메시지와 사용자 메시지를 사용하여 Azure OpenAI API에 요청을 보내고 응답을 반환합니다."""
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=800
    )
    return response.choices[0].message.content

# 기술 전문가 시스템 메시지
tech_system_message = """당신은 Microsoft Azure 전문가입니다. 항상 기술적으로 정확한 정보를 제공하며, 
복잡한 개념도 초보자가 이해할 수 있도록 쉽게 설명합니다. 가능한 경우 코드 예제나 구체적인 단계를 포함해주세요."""

user_question = "Azure Functions와 Azure Logic Apps의 차이점은 무엇인가요?"

print("=== 시스템 메시지 활용 ===\n")
print(get_completion_with_system(tech_system_message, user_question))

=== 시스템 메시지 활용 ===

Azure Functions와 Azure Logic Apps는 둘 다 Azure에서 제공하는 서버리스 컴퓨팅 서비스이지만, 그 목적과 사용 방식이 다릅니다. 아래에서 두 서비스의 주요 차이점에 대해 설명하겠습니다.

### Azure Functions

1. **개발자 중심**: Azure Functions는 개발자가 코드를 작성하고 실행할 수 있는 서버리스 환경입니다. C#, Java, Python, JavaScript 등 다양한 언어를 지원합니다.
   
2. **이벤트 기반**: Azure Functions는 특정 이벤트에 반응하여 실행됩니다. 예를 들어, HTTP 요청, 타이머, Azure Storage Blob 업로드 등 다양한 이벤트를 기반으로 트리거될 수 있습니다.

3. **유연성**: 복잡한 비즈니스 로직을 구현할 수 있으며, 필요한 경우 외부 라이브러리를 추가하여 더 많은 기능을 사용할 수 있습니다.

4. **비용 효율성**: 사용한 만큼만 비용을 지불하는 모델로, 요청 수에 따라 요금이 청구됩니다.

5. **코드 예제**:
   ```csharp
   using System.IO;
   using Microsoft.AspNetCore.Mvc;
   using Microsoft.Azure.WebJobs;
   using Microsoft.Azure.WebJobs.Extensions.Http;
   using Microsoft.AspNetCore.Http;
   using Microsoft.Extensions.Logging;

   public static class FunctionExample
   {
       [FunctionName("HttpExample")]
       public static async Task<IActionResult> Run(
           [HttpTrigger(AuthorizationLevel.Function, "get", "post", Route = null)]

### 4.2. 다중 단계 지시사항 (Multi-Step Instructions)

복잡한 작업을 여러 단계로 나누어 수행하도록 지시할 수 있습니다.

In [12]:
multi_step_prompt = """Azure Web App을 사용하여 간단한 웹 애플리케이션을 배포하는 과정을 알려주세요.
다음 순서로 설명해주세요:

1. 필요한 사전 요구 사항 나열
2. Azure Portal에서 Web App 리소스 생성 단계
3. VS Code를 사용하여 로컬에서 웹 앱 개발하기
4. Azure에 웹 앱 배포하기
5. 배포 후 모니터링 및 문제 해결 방법

각 단계마다 구체적인 명령어나 설정 옵션을 포함해주세요."""

print("=== 다중 단계 지시사항 ===\n")
print(get_completion(multi_step_prompt))

=== 다중 단계 지시사항 ===

Azure Web App을 사용하여 간단한 웹 애플리케이션을 배포하는 과정은 다음과 같습니다.

### 1. 필요한 사전 요구 사항 나열

- **Azure 계정**: Azure Portal에 접근할 수 있는 유효한 계정이 필요합니다. 계정이 없다면 [Azure 무료 계정](https://azure.microsoft.com/free/)으로 가입할 수 있습니다.
- **Visual Studio Code**: 웹 애플리케이션 개발을 위한 코드 편집기입니다. [VS Code 다운로드](https://code.visualstudio.com/)에서 설치합니다.
- **Azure CLI**: Azure 리소스를 관리하기 위해 필요한 커맨드라인 도구입니다. [Azure CLI 설치 가이드](https://docs.microsoft.com/cli/azure/install-azure-cli)에서 설치합니다.
- **Node.js (선택적)**: JavaScript 기반의 웹 애플리케이션을 개발할 경우 필요합니다. [Node.js 다운로드](https://nodejs.org/)에서 설치합니다.

### 2. Azure Portal에서 Web App 리소스 생성 단계

1. **Azure Portal 접속**: [Azure Portal](https://portal.azure.com/)에 로그인합니다.
2. **리소스 생성**: 왼쪽 메뉴에서 "리소스 만들기" 클릭 후, "Web App"을 검색하여 선택합니다.
3. **Web App 설정**:
   - **구독**: 사용할 구독 선택.
   - **리소스 그룹**: 새 리소스 그룹 생성 또는 기존 리소스 그룹 선택.
   - **웹앱 이름**: 고유한 이름 입력 (예: `mywebapp123`).
   - **배포 옵션**: 사용할 플랫폼 선택 (Windows 또는 Linux).
   - **런타임 스택**: 사용할 프로그래밍 언어 및 버전 선택 (예: Node.js, .NET, PHP 등).


### 4.3. 출력 형식 지정 (Output Formatting)

LLM의 응답을 특정 형식으로 받을 수 있습니다.

In [17]:
format_prompt = """Azure의 주요 AI 서비스 5가지를 JSON 형식으로 제공해주세요. 각 서비스에 대해 다음 정보를 포함해주세요:
- 서비스 이름
- 간단한 설명
- 주요 기능 (3가지)
- 일반적인 사용 사례 (2가지)
- 관련 Azure 서비스 (2가지)

응답은 다음과 같은 JSON 형식이어야 합니다:
{
  "ai_services": [
    {
      "name": "서비스 이름",
      "description": "설명",
      "key_features": ["기능1", "기능2", "기능3"],
      "use_cases": ["사용 사례1", "사용 사례2"],
      "related_services": ["관련 서비스1", "관련 서비스2"]
    },
    ...
  ]
}"""

json_response = get_completion(format_prompt)
print("=== 출력 형식 지정 (JSON) ===\n")
print(json_response)

# JSON 응답을 파싱하여 구조화된 데이터로 사용 가능
try:
    parsed_response = json.loads(json_response)
    print("\n=== 첫 번째 서비스 정보 ===\n")
    print(f"서비스 이름: {parsed_response['ai_services'][0]['name']}")
    print(f"설명: {parsed_response['ai_services'][0]['description']}")
    print("주요 기능:")
    for feature in parsed_response['ai_services'][0]['key_features']:
        print(f"- {feature}")
except json.JSONDecodeError as e:
    print(f"JSON 파싱 오류: {e}")

=== 출력 형식 지정 (JSON) ===

```json
{
  "ai_services": [
    {
      "name": "Azure Cognitive Services",
      "description": "다양한 AI 기능을 제공하는 API 및 서비스 모음으로, 이미지 인식, 언어 처리, 음성 인식 등을 포함합니다.",
      "key_features": ["이미지 및 얼굴 인식", "자연어 처리", "음성 인식 및 합성"],
      "use_cases": ["고객 지원 챗봇 구축", "이미지 분석 및 분류"],
      "related_services": ["Azure Machine Learning", "Azure Bot Services"]
    },
    {
      "name": "Azure Machine Learning",
      "description": "데이터 과학자와 개발자가 AI 모델을 구축, 학습 및 배포할 수 있도록 지원하는 플랫폼입니다.",
      "key_features": ["자동화된 머신러닝", "모델 관리 및 배포", "데이터 준비 및 분석 도구"],
      "use_cases": ["예측 분석 모델 개발", "사용자 맞춤형 추천 시스템 구축"],
      "related_services": ["Azure Databricks", "Azure Data Lake Storage"]
    },
    {
      "name": "Azure Bot Services",
      "description": "지능형 봇을 개발하고 배포할 수 있는 서비스로, 다양한 채널과 통합할 수 있습니다.",
      "key_features": ["다양한 채널 지원", "자연어 처리 통합", "봇 프레임워크"],
      "use_cases": ["고객 상담 지원 봇", "주문 처리 및 정보 제공 봇"],
      "related_services": ["Azure Cognitive Services", "A

## 5. 실전 프롬프트 엔지니어링

실제 업무에서 활용할 수 있는 프롬프트 엔지니어링 예제를 살펴봅시다.

### 5.1. Azure 아키텍처 설계 조언 받기

In [18]:
architecture_prompt = """당신은 수석 Azure 솔루션 아키텍트입니다. 다음 요구사항에 맞는 Azure 아키텍처를 설계해주세요:

요구사항:
- 고가용성 웹 애플리케이션 (ASP.NET Core)
- 실시간 데이터 처리 (IoT 디바이스에서 수집된 센서 데이터)
- 대용량 데이터 저장 및 분석
- AI 기반 예측 모델을 통한 이상 징후 감지
- 철저한 보안 및 규정 준수 (개인정보 포함)
- 비용 효율적인 설계

다음 사항을 포함해주세요:
1. 아키텍처 다이어그램 (텍스트로 표현)
2. 각 Azure 서비스의 선택 이유
3. 확장성 및 장애 조치 전략
4. 예상 비용 최적화 방안
5. 보안 고려사항"""

print("=== Azure 아키텍처 설계 조언 ===\n")
print(get_completion(architecture_prompt))

=== Azure 아키텍처 설계 조언 ===

다음은 요구사항에 맞춘 Azure 아키텍처 설계입니다.

### 1. 아키텍처 다이어그램 (텍스트로 표현)

```
+---------------------+
|     IoT Devices     |
+---------------------+
           |
           v
+---------------------+
| Azure IoT Hub       |  <--- IoT 디바이스의 센서 데이터 수집
+---------------------+
           |
           v
+---------------------+
| Azure Stream Analytics| <--- 실시간 데이터 처리 및 분석
+---------------------+
           |
           v
+---------------------+
| Azure Data Lake     | <--- 대용량 데이터 저장
+---------------------+
           |
           v
+---------------------+
| Azure Databricks    | <--- 데이터 분석 및 AI 모델 학습
+---------------------+
           |
           v
+---------------------+
| Azure Machine Learning| <--- AI 기반 예측 모델 생성
+---------------------+
           |
           v
+---------------------+
| Azure App Service    | <--- 고가용성 ASP.NET Core 웹 애플리케이션
+---------------------+
           |
           v
+---------------------+
| Azure SQL Database   | <--- 메타데이터 및 사용자 정보 저장
+-------

### 5.2. Azure CLI 명령어 생성

In [19]:
cli_prompt = """Azure CLI 명령어를 사용하여 다음 작업을 수행하는 스크립트를 생성해주세요:

1. 리소스 그룹 생성 (이름: rg-workshop-prod, 위치: Korea Central)
2. Azure Kubernetes Service(AKS) 클러스터 생성 (3개 노드, 시스템 할당 관리 ID 사용)
3. Azure Container Registry(ACR) 생성 및 AKS와 연결
4. Azure Key Vault 생성 및 필요한 비밀 저장 (DB 연결 문자열, API 키)
5. Azure Monitor를 사용하여 AKS 클러스터 모니터링 설정

각 명령어에 대한 간단한 설명을 주석으로 추가해주세요."""

print("=== Azure CLI 명령어 생성 ===\n")
print(get_completion(cli_prompt))

=== Azure CLI 명령어 생성 ===

다음은 Azure CLI를 사용하여 요청한 작업을 수행하는 스크립트입니다. 각 명령어에는 간단한 설명이 포함되어 있습니다.

```bash
#!/bin/bash

# 1. 리소스 그룹 생성 (이름: rg-workshop-prod, 위치: Korea Central)
az group create --name rg-workshop-prod --location koreaCentral
# 리소스 그룹을 생성하여 나머지 리소스들을 그룹화합니다.

# 2. Azure Kubernetes Service(AKS) 클러스터 생성 (3개 노드, 시스템 할당 관리 ID 사용)
az aks create --resource-group rg-workshop-prod --name aks-cluster --node-count 3 --enable-managed-identity --location koreaCentral
# AKS 클러스터를 생성하고 3개의 노드를 설정하며 시스템 할당 관리 ID를 사용합니다.

# 3. Azure Container Registry(ACR) 생성 및 AKS와 연결
az acr create --resource-group rg-workshop-prod --name acrworkshopregistry --sku Basic
# ACR을 생성하여 컨테이너 이미지를 저장합니다.

# AKS와 ACR 연결
az aks update --name aks-cluster --resource-group rg-workshop-prod --attach-acr acrworkshopregistry
# AKS 클러스터에 ACR을 연결하여 AKS가 ACR의 이미지를 사용할 수 있도록 설정합니다.

# 4. Azure Key Vault 생성 및 필요한 비밀 저장 (DB 연결 문자열, API 키)
az keyvault create --name kv-workshop-prod --resource-group rg-workshop-prod --location

## 6. 실습: 나만의 프롬프트 작성하기

이제 지금까지 배운 내용을 바탕으로 나만의 프롬프트를 작성해봅시다.

In [21]:
# 여기에 직접 프롬프트를 작성하고 실행해보세요
my_prompt = """
# 여기에 프롬프트를 작성하세요
"""

# 주석을 제거하고 실행하세요
# print(get_completion(my_prompt))

## 7. 마무리 및 다음 단계

이 세션에서는 LLM과 프롬프트 엔지니어링의 기본 및 고급 기법을 학습했습니다. 다음 세션에서는 GitHub Copilot과 AutoGen을 활용하여 Multi AI Agent 앱을 구현해볼 예정입니다.

### 학습한 내용 요약
- Azure AI Foundry에서 LLM 모델 배포 및 사용 방법
- 기본 프롬프트 엔지니어링 기법 (명확한 지시사항, 역할 부여, 단계별 사고 유도, Few-Shot 학습)
- 고급 프롬프트 엔지니어링 기법 (시스템 메시지, 다중 단계 지시사항, 출력 형식 지정)
- 실전 프롬프트 엔지니어링 사례

### 추가 학습 자료
- [Azure OpenAI Service 공식 문서](https://docs.microsoft.com/azure/cognitive-services/openai/)
- [Azure AI Foundry 공식 문서](https://docs.microsoft.com/azure/ai-services/)